In [ ]:
#! pip install madgrad
#! pip install efficientnet_pytorch

In [ ]:
import copy
from numpy import asarray
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
import random
import math
from torch.autograd import Variable
#import matplotlib.pyplot as plt
#from modules import *
#import torchvision.models as models_pytorch
#import h5py
#import torch.optim as optim
#import augmentations
from torch.nn.functional import kl_div, softmax, log_softmax
import torch.optim as optim
import torch.utils.data as data
from os.path import exists, join, split
from os import listdir
from os.path import join
from PIL import Image, ImageFilter , ImageDraw
import PIL
import random
#import madgrad 
import matplotlib.pyplot as plt



In [ ]:
num_class = 10 #number of classes
total_steps = 2**20
eval_step = 1024
threshhold = 0.95
num_epochs = math.ceil(total_steps/eval_step)
seed=111 #seed for the algorithm
batch_size = 32
labeled_batch_size = 64
u=7
unlabeled_batch_size = labeled_batch_size*u
num_train =100 # number of training image by classe
#Validation set size
valid_size = 200


## First let us define a CNN

In [ ]:


bn_momentum = 0.9


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)


def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)


class WideBasic(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1):
        super(WideBasic, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=bn_momentum)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(planes, momentum=bn_momentum)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)

        return out


class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(WideResNet, self).__init__()
        self.in_planes = 16

        assert ((depth - 4) % 6 == 0), 'Wide-resnet depth should be 6n+4'
        n = int((depth - 4) / 6)
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3, nStages[0])
        self.layer1 = self._wide_layer(WideBasic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(WideBasic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(WideBasic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=bn_momentum)
        self.linear = nn.Linear(nStages[3], num_classes)

        # self.apply(conv_init)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        # out = F.avg_pool2d(out, 8)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out

class WideResNet_NDA(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(WideResNet_NDA, self).__init__()
        self.in_planes = 16

        assert ((depth - 4) % 6 == 0), 'Wide-resnet depth should be 6n+4'
        n = int((depth - 4) / 6)
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3, nStages[0])
        self.layer1 = self._wide_layer(WideBasic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(WideBasic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(WideBasic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=bn_momentum)
        self.linear = nn.Linear(nStages[3], num_classes)

        # self.apply(conv_init)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        # out = F.avg_pool2d(out, 8)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        logit = self.linear(out)

        return out,logit




class WideResNet_conf(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(WideResNet_conf, self).__init__()
        self.in_planes = 16

        assert ((depth - 4) % 6 == 0), 'Wide-resnet depth should be 6n+4'
        n = int((depth - 4) / 6)
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3, nStages[0])
        self.layer1 = self._wide_layer(WideBasic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(WideBasic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(WideBasic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=bn_momentum)
        self.linear = nn.Linear(nStages[3], num_classes)
        self.fconfid =  nn.Linear(nStages[3], 1)

        # self.apply(conv_init)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        # out = F.avg_pool2d(out, 8)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out_final = self.linear(out)
        out_conf = self.fconfid(out)

        return out_final,out_conf


class WideResNet_drop(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(WideResNet_drop, self).__init__()
        self.in_planes = 16

        assert ((depth - 4) % 6 == 0), 'Wide-resnet depth should be 6n+4'
        n = int((depth - 4) / 6)
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3, nStages[0])
        self.layer1 = self._wide_layer(WideBasic, nStages[1], n, dropout_rate, stride=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.layer2 = self._wide_layer(WideBasic, nStages[2], n, dropout_rate, stride=2)
        self.dropout2 = nn.Dropout2d(0.25)
        self.layer3 = self._wide_layer(WideBasic, nStages[3], n, dropout_rate, stride=2)
        self.dropout3 = nn.Dropout2d(0.25)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=bn_momentum)
        self.dropout4 = nn.Dropout2d(0.25)
        self.linear = nn.Linear(nStages[3], num_classes)

        # self.apply(conv_init)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.dropout1(out)
        out = self.layer2(out)
        out = self.dropout2(out)
        out = self.layer3(out)
        out = self.dropout3(out)
        out = F.relu(self.bn1(out))
        # out = F.avg_pool2d(out, 8)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.dropout4(out)
        out = self.linear(out)
        

        return out

In [ ]:
! pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
def loadEffNet(pretrained=False, b=0):
    model  = None
    if(pretrained):
        model = EfficientNet.from_pretrained('efficientnet-b{}'.format(b))
        for param in model.parameters():
            param.requires_grad = False
    else:
        model =  EfficientNet.from_name('efficientnet-b{}'.format(b))
    model._fc = nn.Linear(1280,10, bias=True)
    model = model.cuda()
    return model

## now let us define a dataset

In [ ]:
from PIL import Image, ImageEnhance
import random

def AddBlur(img, factor):
    return img.filter(ImageFilter.GaussianBlur(factor))

def AddBrightness(img,factor):
    factor=max(factor,0.4)
    return ImageEnhance.Brightness(img).enhance(factor)

def AddColor(img,factor):
    return ImageEnhance.Color(img).enhance(factor)

def AddContrast(img,factor):
    return ImageEnhance.Contrast(img).enhance(factor)

def AddCutout(img,length):
    w, h = img.size
    x0 = np.random.uniform(0, w)
    y0 = np.random.uniform(0, h)
    x0 = int(max(0, x0 - length / 2.))
    y0 = int(max(0, y0 - length / 2.))
    x1 = int(min(w, x0 + length))
    y1 = int(min(h, y0 + length))
    xy = (x0, y0, x1, y1)
    # gray
    color = (127, 127, 127)
    img = img.copy()
    ImageDraw.Draw(img).rectangle(xy, color)
    return img


def AddRotation(img,factor):
    return img.rotate(factor)
    
def AddTranslationX(img,factor):
    if random.random() > 0.5:
        factor = -factor
    factor = factor * img.size[0]
    return img.transform(img.size, PIL.Image.AFFINE, (1,0,factor,0,1,0))

def AddTranslationY(img,factor):
    if random.random() > 0.5:
        factor = -factor
    factor = factor * img.size[0]
    return img.transform(img.size, PIL.Image.AFFINE, (1,0,0,0,1,factor))

class AddRandAugment:
    def __init__(self,n,m):
        assert n >= 1
        assert 1 <= m <= 10
        self.n = n
        self.m = m
        self.augmentations = AugmentationsPossibilities()
    
    def __call__(self,img):
        choices = random.choices(self.augmentations,k=self.n)
        for (aug,factor) in choices:
            frac = random.randint(1,self.m)/self.m
            img = aug(img,frac*factor)
        return img

def AugmentationsPossibilities():
    res = [(AddBrightness,1.2),
          (AddContrast,1.8),
          (AddCutout,8),
          (AddRotation,15),
          (AddTranslationX,0.3),
          (AddTranslationY,0.3),
          (AddBlur,0.8)]
    return res

class TransformAug:
    def __init__(self,mean,std):
        self.weak = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(size=32,padding=int(32*0.125),
                                  padding_mode='reflect'),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)])   
        self.strong = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(size=32,
                                  padding=int(32*0.125),
                                  padding_mode='reflect'),
            AddRandAugment(n=2, m=10),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)])
        
    def __call__(self,img):
        return self.weak(img),self.strong(img)
            

In [ ]:
class Dataset_sub_CIFAR(data.Dataset):

    def __init__(self, data_feature, data_target,transform,phase='label'):
        self.data_feature = data_feature
        self.data_target = data_target
        self.transform = transform
        self.phase=phase



    def __len__(self):
        return len(self.data_feature)

    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        if self.phase=='label':
            data_feature = self.transform(Image.fromarray(np.uint8(self.data_feature[index])))
            data_target =  self.data_target[index]
            return data_feature, data_target

        else:
            data_feature = self.data_feature[index].float()
            return data_feature


data_mean = (0.4914, 0.4822, 0.4465)
data_std = (0.2471, 0.2435, 0.2616)

transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=32,
                             padding=int(32*0.125),
                             padding_mode='reflect'),
        transforms.ToTensor(),
        transforms.Normalize(mean=data_mean, std=data_std)]
    )
transform_train_Aug = TransformAug(mean=data_mean, std=data_std)
transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=data_mean, std=data_std)
    ])
transform_test=TransformAug(mean=data_mean, std=data_std)
    
    #Dataset loading
CIFAR10_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=None, download=True)
CIFAR10_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=None, download=True)
np.random.seed(seed=seed)
permuation=np.random.permutation(len(CIFAR10_train_dataset.targets))

Original_train_data_x = (CIFAR10_train_dataset.data)
Original_train_data_y = np.array(CIFAR10_train_dataset.targets)
Original_train_data_x = Original_train_data_x[permuation]
Original_train_data_y = Original_train_data_y[permuation]

Original_test_data_x = CIFAR10_test_dataset.data
Original_test_data_y = np.array(CIFAR10_test_dataset.targets)



    #Selection of 250 labeled images for training and 2000 for validation
    
incr_class = torch.zeros(num_class)
train_idx_dico = {} #labeled images index dictionnary

for i in range(num_class):
    train_idx_dico[str(i)] = []

valid_idx = np.zeros(num_class * valid_size, dtype=np.int32) #validation images indexes (2000)
incr_t = 0
incr_v = 0
incrtotal = 0

for idx in range(len(Original_train_data_y)):
    class_y = Original_train_data_y[idx]
    incrtotal += 1

    train_idx_dico[str(class_y)].append(idx)
    incr_class[class_y] += 1 #count the number of image per class
    incr_t += 1


train_idx = np.zeros(num_class * num_train, dtype=np.int32) #train labeled images indexes (250)
list_train_id = []
list_unalabel_id = []
valid_idx = []
unlabel_idx_dico = {}
for i in range(num_class):
    unlabel_idx_dico[str(i)] = []
for i in range(num_class):
    list_train_id = list_train_id + train_idx_dico[str(i)][0:num_train]
    valid_idx =valid_idx + train_idx_dico[str(i)][num_train:num_train+valid_size]
    list_unalabel_id = list_unalabel_id + train_idx_dico[str(i)][num_train+valid_size::]
    unlabel_idx_dico[str(i)] = train_idx_dico[str(i)][num_train::]

    #Get labeled and unlabeled data

x_train = Original_train_data_x[[int(i) for i in list_train_id]]
y_train = Original_train_data_y[[int(i) for i in list_train_id]]

x_unlabeled = Original_train_data_x[[int(i) for i in list_unalabel_id]]
y_unlabeled = Original_train_data_y[[int(i) for i in list_unalabel_id]]

    #Get validation set data
x_valid = Original_train_data_x[[int(i) for i in valid_idx]]
y_valid = Original_train_data_y[[int(i) for i in valid_idx]]

    # Printing the size of the training, validation and test sets
print('Number of training examples: ' + str(x_train.shape[0]))
print('Number of unlabeled examples: ' + str(x_unlabeled.shape[0]))
print('Number of validation examples: ' + str(x_valid.shape[0]))

classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    #Dataloader creation

test_loader = torch.utils.data.DataLoader(
        Dataset_sub_CIFAR(Original_test_data_x, Original_test_data_y, transform=transform_test),
        batch_size = unlabeled_batch_size,
        shuffle=False, num_workers=2)

train_loader = torch.utils.data.DataLoader(
        Dataset_sub_CIFAR(x_train, y_train, transform=transform_train_Aug),
        batch_size=labeled_batch_size,shuffle=True, num_workers=2) #num_workers = 2 ou 1

valid_loader = torch.utils.data.DataLoader(
        Dataset_sub_CIFAR(x_valid, y_valid, transform=transform_val),
        batch_size=batch_size,
        shuffle=False, num_workers=2)
    
final_loader = torch.utils.data.DataLoader(
        Dataset_sub_CIFAR(Original_test_data_x, Original_test_data_y, transform=transform_val),
        batch_size = 64,
        shuffle=False, num_workers=2) 

In [ ]:
def showDiffAug(img,m):
    choices = AugmentationsPossibilities()
    fig, axs = plt.subplots(len(choices),m, figsize = (20,20))
    j=0
    for (aug,factor) in choices:
        for i in range(1,m+1):
            frac = i/m
            newimg = aug(img,frac*factor)
            newimg = asarray(newimg)
            axs[j][i-1].imshow(newimg)
            axs[j][i-1].set_title(aug.__name__+" "+str(i))
        j+=1
img =Image.fromarray(Original_train_data_x[1])
showDiffAug(img,10)

## Now we build the CNN and the optimizer


In [ ]:
from math import cos, pi
def learning_rate_scheduler(init, epoch , num_epochs):
    return init*cos(7*pi*epoch/(16*num_epochs))

    return init*math.pow(0.1, optim_factor)
def getlambda(epoch, trainacc):
    return 1
    """if(trainacc<0.8):
        return 0
    if epoch<=50 : 
        return 0
    if epoch<=100:
        return 1
    if epoch<=150:
        return 2
    if epoch<=200:
        return 3
    if epoch<=250:
        return 4
    return 5"""

def test(epoch,net,testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))
    return acc

        

In [ ]:
criterion = nn.CrossEntropyLoss()
pretrained = False
#Networks creation
initialnet = WideResNet(28,2, dropout_rate=0.2, num_classes=num_class)
initialnet = initialnet.cuda()
initialnet_save = WideResNet(28, 2, dropout_rate=0.2, num_classes=num_class) # model where to save the results
initialnet_save = initialnet_save.cuda()

In [ ]:
net = copy.deepcopy(initialnet_save)
net_save = copy.deepcopy(initialnet_save)

In [ ]:
best_acc=0
log_interval = 15
labeled_iter = iter(train_loader)
besttrainacc=0
unlabeled_iter = iter(test_loader)
lr = 0.03
for epoch in range(num_epochs):  
    net.train()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate_scheduler(lr, epoch, num_epochs), momentum=0.9, weight_decay=5e-4, nesterov=True)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate_scheduler(lr, epoch,num_epochs)))
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx in range(eval_step):
        try : 
            (x_t_w,x_t_s), y = labeled_iter.next()
        except:
            labeled_iter = iter(train_loader)
            (x_t_w,x_t_s), y = labeled_iter.next()
        try : 
            (x_w, x_s), _ = unlabeled_iter.next()
        except:
            unlabeled_iter = iter(test_loader)
            (x_w, x_s), _ = unlabeled_iter.next()
        inputs = torch.cat((x_t_w,x_t_s,x_w,x_s)).cuda()
        optimizer.zero_grad()
        outputs = net(inputs) 
        y_outputs = outputs[:len(x_t_w)+len(x_t_s)]
        #print(len(x))
        y_w , y_s = outputs[len(x_t_w)+len(x_t_s):len(x_t_w)+len(x_t_s)+len(x_w)], outputs[len(x_t_w)+len(x_t_s)+len(x_w):]
        y=y.cuda()
        y = torch.cat((y,y)).cuda()
        lossL = criterion(y_outputs, y)  # Loss
        pseudo_labels = torch.softmax(y_w,dim=-1)
        prob,pseudo_labels = torch.max(pseudo_labels,dim=-1)
        prob_mask = (prob >=threshhold).float()
        lossU =(criterion(y_s,pseudo_labels) * prob_mask).mean()
        _, predicted = torch.max(y_outputs.data, 1)
        _, pseudopredicted = torch.max(y_s,1)
        pseudopredicted=(pseudopredicted*prob_mask)
        total += y.size(0)
        totalpseudo = y_s.size(0)
        correct += predicted.eq(y.data).cpu().sum() 
        correctpseudo = 0
        besttrainacc = max((besttrainacc,correct/total))
        correctpseudo=(pseudopredicted).eq(pseudo_labels*prob_mask).cpu().sum()
        
        loss = lossL+ getlambda(epoch, besttrainacc)*lossU
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update
        train_loss += loss.item()
        if(getlambda(epoch, besttrainacc)==0):
            correctpseudo=0
        if batch_idx % log_interval == 0:
            print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%% \t - \t pseudoAcc: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (eval_step), loss.item(), 100.*correct/total, 100.*correctpseudo/totalpseudo))
    acc =test(epoch,net,valid_loader)
    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
        net_save.load_state_dict(net.state_dict(), strict=True)
        best_acc=acc



In [ ]:
ssl_net = copy.deepcopy(net_save)

In [ ]:
import pandas as pd


def test_final(net,testloader):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs_weak, targets) in enumerate(testloader):
            
            inputs_weak, targets = inputs_weak.cuda(), targets.cuda()
            inputs_weak, targets = Variable(inputs_weak), Variable(targets)
            outputs = net(inputs_weak)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()
            if batch_idx == 0:
                predicted_concat = predicted.clone()
            else:
                predicted_concat = torch.cat((predicted_concat, predicted), 0)

        # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| TEST \t\t\tLoss: %.4f Acc@1: %.2f%%" %( loss.item(), acc))
    return predicted_concat.cpu().numpy()
    

predicted_concat = test_final(ssl_net,final_loader)


id_concat =range(len(predicted_concat))
my_submission = pd.DataFrame({'Id': id_concat,'Expected': predicted_concat})

# you could use any filename. We choose submission here
my_submission.to_csv('submission2.csv', index=False)
print('we have saved the submission !! ')
    